# Step 1: Import Required Libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import joblib

# Step 2: Load the Dataset

In [2]:
# Load dataset
df = pd.read_csv(r"C:\Users\hp\OneDrive\Desktop\MARATHON\HAIR_FALL_PREDICT.csv")

# Step 3: Preprocess the Data

In [3]:
# Remove Unnecessary Columns
df.drop(columns=["Unnamed: 0", "Timestamp", "What_is_your_name"], inplace=True, errors="ignore")

# Clean Column Names
df.columns = (df.columns.str.strip()
              .str.replace(r"[?]", "", regex=True)  # Remove '?'
              .str.replace(r"[ ,]", "_", regex=True)  # Replace spaces with '_'
              .str.replace(r"_+", "_", regex=True)  # Remove multiple '_'
              .str.rstrip("_"))  # Remove trailing '_'

# Convert Yes/No columns to binary (1/0)
binary_columns = [
    "Do_you_have_hair_fall_problem",
    "Is_there_anyone_in_your_family_having_a_hair_fall_problem_or_a_baldness_issue",
    "Did_you_face_any_type_of_chronic_illness_in_the_past",
    "Do_you_stay_up_late_at_night",
    "Do_you_have_any_type_of_sleep_disturbance",
    "Do_you_think_that_in_your_area_water_is_a_reason_behind_hair_fall_problems",
    "Do_you_use_chemicals_hair_gel_or_color_in_your_hair",
    "Do_you_have_anemia",
    "Do_you_have_too_much_stress"
]

df[binary_columns] = df[binary_columns].apply(lambda x: x.map({"Yes": 1, "No": 0}))

# Initialize label encoders dictionary
label_encoders = {}

# List of categorical columns to encode (Check if they exist before encoding)
categorical_columns = ["What_is_your_gender", "What_is_your_food_habit"]

# Apply label encoding only to existing columns
for col in categorical_columns:
    if col in df.columns:  # Check if column exists
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])  # Encode column
        label_encoders[col] = le  # Store encoder for future use
    else:
        print(f"Skipping encoding: Column '{col}' not found in dataset.")

# Normalize Age Column
scaler = MinMaxScaler()
df["What_is_your_age"] = scaler.fit_transform(df[["What_is_your_age"]])

# Drop Any Remaining Non-Numeric Columns
df = df.select_dtypes(exclude=['object'])

# Handle Missing Values
df.fillna(df.median(), inplace=True)
df.head()



,What_is_your_age,What_is_your_gender,Do_you_have_hair_fall_problem,Is_there_anyone_in_your_family_having_a_hair_fall_problem_or_a_baldness_issue,Did_you_face_any_type_of_chronic_illness_in_the_past,Do_you_stay_up_late_at_night,Do_you_have_any_type_of_sleep_disturbance,Do_you_think_that_in_your_area_water_is_a_reason_behind_hair_fall_problems,Do_you_use_chemicals_hair_gel_or_color_in_your_hair,Do_you_have_anemia,Do_you_have_too_much_stress,What_is_your_food_habit
0,0.034483,1,1,1,1,1,0,0,0.0,1,1.0,0
1,0.014778,1,0,0,0,0,0,1,1.0,0,0.0,0
2,0.049261,1,1,1,1,0,0,1,0.0,0,1.0,0
3,0.009852,1,0,0,0,1,0,1,0.0,1,1.0,0
4,0.064039,1,0,1,1,1,0,1,0.0,1,1.0,2


# Step 4: Build and Train the Neural Network Model

In [4]:
# Define features (X) and target variable (y)
X = df.drop(columns=["Do_you_have_hair_fall_problem"])  # Features
y = df["Do_you_have_hair_fall_problem"]  # Target variable

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print shape of training data
X_train.shape, X_test.shape

# Build a simple neural network model
model = keras.Sequential([
    keras.layers.Dense(16, activation="relu", input_shape=(X_train.shape[1],)),
    keras.layers.Dense(8, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")  # Sigmoid for binary classification
])

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Display model summary
model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=8, validation_data=(X_test, y_test))


C:\Users\hp\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 16)             │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 337 (1.32 KB)

 Trainable params: 337 (1.32 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.4182 - loss: 0.7608 - val_accuracy: 0.7361 - val_loss: 0.5587
Epoch 2/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7130 - loss: 0.5380 - val_accuracy: 0.7639 - val_loss: 0.5128
Epoch 3/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7546 - loss: 0.4901 - val_accuracy: 0.7986 - val_loss: 0.5014
Epoch 4/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8073 - loss: 0.4498 - val_accuracy: 0.7917 - val_loss: 0.4944
Epoch 5/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.8077 - loss: 0.4370 - val_accuracy: 0.7917 - val_loss: 0.4936
Epoch 6/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8286 - loss: 0.4097 - val_accuracy: 0.7917 - val_loss: 0.4938
Epoch 7/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8556 - loss: 0.3770 - val_accuracy: 0.7917 - val_loss: 0.4905
Epoch 8/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8485 - loss: 0.3679 - val_accuracy: 0.7986 - val_l

# Step 5: Save the Model

In [5]:
# Save the trained model as a .h5 file
model.save("hairfall_model.h5")
print("Model saved as hairfall_model.h5")


Model saved as hairfall_model.h5


In [6]:
# Save both scaler and encoders in a dictionary
if label_encoders:
    preprocessor = {"scaler": scaler, "encoders": label_encoders}
    joblib.dump(preprocessor, "scaler.pkl")
    print("Preprocessor saved as scaler.pkl")
else:
    preprocessor = {"scaler": scaler}
    joblib.dump(preprocessor, "scaler.pkl")
    print("No categorical encoding applied. Only scaler saved as scaler.pkl.")


Preprocessor saved as scaler.pkl


# step 6: Testing the Model

In [7]:
# Load the saved model
loaded_model = keras.models.load_model("hairfall_model.h5")

# Evaluate model on test data
loss, accuracy = loaded_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7801 - loss: 0.5452  
Test Accuracy: 0.82


# Step 7: Personalized Recommendations

In [2]:
def get_recommendations(features):
    recommendations = []
    
    if features[0] < 25:
        recommendations.append("Ensure adequate protein intake for hair growth.")
    if features[1] == 1 and features[8] == 1:
        recommendations.append("Iron deficiency (Anemia) is a major cause of hair fall. Consider iron-rich foods.")
    if features[9] == 1:
        recommendations.append("Manage stress with meditation and exercise.")
    if features[5] == 1:
        recommendations.append("Improve sleep quality and maintain a consistent sleep schedule.")
    if features[7] == 1:
        recommendations.append("Reduce the use of hair chemicals and switch to natural products.")
    if features[10] == 2:
        recommendations.append("Fast food diets lack essential nutrients. Shift towards a balanced diet.")

    # Additional detailed recommendations (Ensuring valid index range)
    if features[0] < 6:
        recommendations.append('Increase your sleep to at least 6-8 hours per night.')
    if features[1] > 7:
        recommendations.append('Manage stress with meditation or relaxation exercises.')
    if features[2] < 50:
        recommendations.append('Eat protein-rich foods to improve hair strength.')
    if features[3] > 3:
        recommendations.append('Reduce shampoo frequency to avoid dryness.')
    if features[4] == 1:
        recommendations.append('Consider taking biotin or iron supplements as per doctor’s advice.')
    if features[5] == 1:
        recommendations.append('Use dermatologist-recommended hair growth treatments like minoxidil.')
    if features[6] > 30:
        recommendations.append('Stay hydrated and avoid excessive caffeine intake.')
    if features[7] > 5:
        recommendations.append('Massage your scalp with essential oils like coconut or rosemary oil.')
    
    # Additional medical recommendations (Ensuring no out-of-bounds indexing)
    if features[8] == 1:
        recommendations.append('Consult a dermatologist for potential treatments like finasteride or platelet-rich plasma (PRP) therapy.')
    if features[9] == 1:
        recommendations.append('Use prescribed antifungal shampoos if experiencing dandruff or scalp infections.')
    if features[10] == 1:
        recommendations.append('Consider medications such as corticosteroids for severe hair fall, as advised by a doctor.')

    return recommendations
